在这个笔记本中，将演示如何使用 Unsloth 对 DeepSeek-R1-Distill-Llama-8B 进行微调，使用的医疗数据集。

### 为什么我们需要对大语言模型（LLM）进行微调？
微调可以让模型在特定任务上表现得更好，从而在实际应用中更加高效和多样化。这个过程对于将已有模型定制为适应特定任务或领域是非常关键的。

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
!pip install bitsandbytes unsloth_zoo


## Choose a Base Model

1. Choose a model that aligns with your usecase
2. Assess your storage, compute capacity and dataset
3. Select a Model and Parameters
4. Choose Between Base and Instruct Models

In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/root/autodl-tmp/models/DeepSeek",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 03-23 11:21:03 [__init__.py:256] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.50.0. vLLM: 0.8.1.
   \\   /|    NVIDIA GeForce RTX 4090 D. Num GPUs = 1. Max memory: 23.643 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## 在微调前推理测试一下

In [2]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
<think>{}"""

In [3]:
question = "一个患有急性阑尾炎的病人已经发病5天，腹痛稍有减轻但仍然发热，在体检时发现右下腹有压痛的包块，此时应如何处理？"


FastLanguageModel.for_inference(model)
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<think>
嗯，我现在要处理一个急性阑尾炎患者的情况。患者已经有5天的病程，腹痛稍微减轻，但仍然发热。在体检时，医生发现了右下腹有压痛的包块。这让我有点困惑，因为急性阑尾炎通常发生在左下腹，而右下腹的压痛包块可能提示其他疾病。

首先，我需要确认诊断。急性阑尾炎的典型症状是左下腹的阵发性疼痛，伴随发热、恶心、呕吐等。患者的腹痛虽然减轻，但仍有发热，这可能意味着炎症正在缓解，但病情尚未完全控制。此外，右下腹的压痛包块不符合急性阑尾炎的位置，这可能提示腹膜炎、appendicitis on the right side，或者其他如肠梗阻、感染性腹膜炎等。

接下来，我需要考虑是否有误诊的可能性。急性阑尾炎通常不会出现在右侧，所以发现右下腹的包块可能提示其他原因。需要进一步检查，比如平面腹部X光、超声或CT来确定是否有实体病变，如腺体炎、肠梗阻、腹膜炎、附睾炎等。

另外，患者的病程已经5天，腹痛减轻可能意味着炎症开始缓解，但仍需注意是否有并发症，如感染、出血或并发症。包块的存在可能需要排除急性阑尾炎以外的疾病。

因此，处理步骤应包括：
1. 确认诊断，排除急性阑尾炎的可能性，考虑其他原因。
2. 进行进一步的影像学检查，如平面腹部X光、超声或CT，确定包块的性质。
3. 根据检查结果，制定相应的治疗方案，如抗生素治疗、外科干预等。
4. 监测患者的病情，必要时进行调整。

在此过程中，可能需要结合患者的症状、病程、实验室检查结果等信息，综合分析以确定正确的诊断和治疗方案。
</think>

根据患者的情况，以下是处理步骤：

1. **确认诊断**：排除急性阑尾炎的可能性，考虑右下腹压痛包块可能的其他原因，如附睾炎、腹膜炎、肠梗阻等。

2. **进一步检查**：
   - 平面腹部X光：评估包块的大小、位置、边界。
   - 超声或CT：确定包块的具体性质，如是否有实体肿瘤或液体积。

3. **治疗方案**：
   - 根据检查结果，若为附睾炎，考虑抗生素治疗。
   - 若为腹膜炎或肠梗阻，可能需要外科干预。

4. **监测与调整**：持续监测患者病情，根据治疗效果及时调整治疗方案。

通过以上步骤，可以准确诊断并妥善处理患者的病情。<｜end▁of▁sentence｜>


In [4]:
#训练

## Prepare Dataset

A medical dataset [https://huggingface.co/datasets/FreedomIntelligence/medical-o1-reasoning-SFT/](https://huggingface.co/datasets/FreedomIntelligence/medical-o1-reasoning-SFT/) will be used to train the selected model.

In [5]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
<think>
{}
</think>
{}"""

### Important Notice
务必在每条训练数据的末尾添加 EOS（序列结束）标记，否则可能会导致模型生成内容时出现无限循环的问题。

In [6]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN


def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

In [7]:
from datasets import load_dataset
dataset = load_dataset("/root/autodl-tmp/dataset", 'zh', split = "train[0:500]", trust_remote_code=True)
print(dataset.column_names)

['Question', 'Complex_CoT', 'Response']


为了使 Ollama 和 llama.cpp 像一个自定义的 ChatGPT 聊天机器人那样工作，我们的数据集必须只包含两列：一列是 instruction，一列是 output。因此，我们需要将数据集转换为正确的结构。

In [8]:
dataset = dataset.map(formatting_prompts_func, batched = True)
dataset["text"][0]

'Below is an instruction that describes a task, paired with an input that provides further context.\nWrite a response that appropriately completes the request.\nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.\nPlease answer the following medical question.\n\n### Question:\n根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？\n\n### Response:\n<think>\n这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。\n\n用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。\n\n但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。\n\n等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。\n\n仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。\n\n回到基本

In [9]:
dataset

Dataset({
    features: ['Question', 'Complex_CoT', 'Response', 'text'],
    num_rows: 500
})

## Train the model
Now let's use Huggingface TRL's `SFTTrainer`.

In [10]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.18 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [11]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        # num_train_epochs = 1, # For longer training runs!
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [12]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/4,582,543,360 (0.92% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.270300
2,2.349100
3,2.293700
4,2.171700
5,2.074500
6,2.048200
7,2.150200
8,1.921400
9,1.980000
10,1.885000


## Inference after fine-tuning

Let's inference with same question again and see the difference.

In [13]:
print(question)

一个患有急性阑尾炎的病人已经发病5天，腹痛稍有减轻但仍然发热，在体检时发现右下腹有压痛的包块，此时应如何处理？


In [14]:
FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<think>
这个病人已经有五天的急性阑尾炎了，虽然他的腹痛已经稍微减轻了，但他仍然发烧，还在右下腹部有一个压痛的包块。这让我有些困惑，因为我原本以为阑尾炎的症状会随着时间的推移而逐渐减轻。

我想起急性阑尾炎的常见表现，包括发热、腹痛、右下腹压痛、发热等等。虽然他的腹痛有所缓解，但包块的存在让我觉得情况可能并没有完全好转。

也许这是一个急性阑尾炎的重度表现，或者是急性阑尾炎的一种严重形式，比如急性阑尾炎的重度表现或者急性阑尾炎的重度表现。考虑到包块的存在，可能需要进一步的评估和治疗。

我想，包块的出现可能意味着感染的扩展或者某种感染性结构的形成。也许需要进一步的影像学检查，比如超声检查或者CT扫描，以便了解包块的具体性质。

哦，对了，包块的存在可能意味着胰腺炎。急性胰腺炎可能与急性阑尾炎有关，尤其是由于细菌感染引起的急性阑尾炎。这种情况下，包块可能是胰腺的炎症反应。

如果包块是由于急性胰腺炎引起的，那么我们需要考虑急性胰腺炎的处理。通常，急性胰腺炎的治疗包括抗生素、抗酸药物和对症治疗，以缓解疼痛和减轻炎症。

不过，包块的存在可能提示了急性胰腺炎的细菌性感染，或者是急性胰腺炎的一种重度表现。这种情况下，可能需要更积极的治疗措施，比如抗生素的使用和可能的外科干预。

另外，包块的存在可能意味着感染的扩展，可能需要进一步的病理学检查以确认包块的性质。

综上所述，考虑到包块的存在，我认为需要进一步评估患者的病情，进行影像学检查以确定包块的具体性质，并根据病情制定相应的治疗方案。
</think>
根据患者的症状和体检结果，包块的存在可能提示急性阑尾炎的感染扩展，甚至可能是急性胰腺炎。建议进行进一步的影像学检查，如超声或CT扫描，以明确包块的性质。根据检查结果和病情的具体情况，可能需要进行抗生素治疗、抗酸药物治疗以及其他针对性治疗措施。<｜end▁of▁sentence｜>


## Upload Model to HuggingFace

Now, let's save our finetuned model and upload it to HuggingFace.

### Save the fine-tuned model to GGUF format

Choose the llama.cpp's GGUF format we prefer by setting the corresponding `if` to `True`.

In [17]:
import os
HUGGINGFACE_TOKEN = os.getenv("token")


In [18]:
# Save to 8bit Q8_0
if True: model.save_pretrained_gguf("model", tokenizer,)

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model_f16", tokenizer, quantization_method = "f16")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 756.9 out of 1007.54 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [00:00<00:00, 47.46it/s]


Unsloth: Saving tokenizer... Done.
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at model into q8_0 GGUF format.
The output location will be /root/autodl-tmp/model/unsloth.Q8_0.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Set model parameters
INFO:hf-to-gguf:gguf: context length = 131072
INFO:hf-to-gguf:gguf: embedding length = 4096
INFO:hf-to-gguf:gguf: feed forward length = 14336
INFO:hf-to-gguf:gguf: head count = 32
INFO:hf-to-gguf:gguf: key-value head count = 8
INFO:hf-to-gguf:gguf: rop

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Unsloth: Conversion completed! Output location: /root/autodl-tmp/model/unsloth.Q8_0.gguf


### Push the model to HuggingFace

Create a model type repository for your model if you haven't done so.

In [19]:
from huggingface_hub import create_repo
create_repo("qwqqwq/medical-model", token=HUGGINGFACE_TOKEN, exist_ok=True)

ConnectTimeout: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /api/repos/create (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fef2978a320>, 'Connection to huggingface.co timed out. (connect timeout=None)'))"), '(Request ID: f3fa6279-2c18-48a2-b39d-98695ec69b33)')

In [ ]:
model.push_to_hub_gguf("qwqqwq/medical-model", tokenizer, token = HUGGINGFACE_TOKEN)

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 33.04 out of 52.96 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [00:02<00:00, 13.40it/s]


Unsloth: Saving tokenizer... Done.
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at wyang14/medical-model into q8_0 GGUF format.
The output location will be /content/wyang14/medical-model/unsloth.Q8_0.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: medical-model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-0

  0%|          | 0/1 [00:00<?, ?it/s]

unsloth.Q8_0.gguf:   0%|          | 0.00/8.54G [00:00<?, ?B/s]

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Saved GGUF to https://huggingface.co/wyang14/medical-model


### Ollama run HuggingFace model

```bash
ollama run hf.co/{username}/{repository}:{quantization}
```

### Ollama inference

```bash
curl http://localhost:11434/api/chat -d '{ \
  "model": "", \
  "messages": [ \
    { "role": "user", "content": "一个患有急性阑尾炎的病人已经发病5天，腹痛稍有减轻但仍然发热，在体检时发现右下腹有压痛的包块，此时应如何处理？" } \
  }'
```